# ICMEd Computational Kinetics Module

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt

## Problem 2

A $7~\mathrm{\mu m}$ thick B-doped (p-type) Si wafer is annealed at $950~\mathrm{{}^\circ C}$ while in equilibrium with a gas containing P vapor (donor).  The concentration of B in the wafer is $2\times 10^{17} / \mathrm{cm^3}$.  P diffusivity in this system at this condition is $D\approx 10^{-14}~\mathrm{cm^2/s}$.  Assume that the concentration of P at the surface in this case is $10^{21}/\mathrm{cm^3}$.  To fabricate a device with a junction at approximately $1~\mathrm{\mu m}$ from the surface, one would like to match the P and B concentrations at a depth of $1~\mathrm{\mu m}$.  

NOTE: 
 - Distances are in micrometers
 - Diffusion coefficient DP is in $\mathrm{\mu m}^2/s$
 - Times are in seconds

In [ ]:
DP = 1.0e-6 # Diffusion coefficient (mum^2 / s)
cP = 1.0e9  # Surface concentration of phosphorus (mum^-3)
cB = 2.0e5  # Bulk concentration of boron (mum^-3)

OutputDist = 1.0    # Distance at which we will output concentration (mum)
ThickM  = 7.0       # thickness of wafer (mum)

Part of the Si surface is covered by a mask in the manner shown below.
![](img/schematic2D.png)

In [ ]:
LengthM = 19.0      # Length of device
mask = 5.0          # width of mask opening

## (a)
How would you set up the domain of the simulation?  What boundary conditions would you impose and where?

In [ ]:
import fipy as fp
from IPython.display import display, clear_output

In [ ]:
gridThickM = 1.0e-1 # grid size (mum)
nx = int(LengthM/gridThickM) # number of grid points parallel to surface
ny = int(ThickM/gridThickM) # number of grid points perpendicular to surface

In [ ]:
mesh = fp.Grid2D(dx=gridThickM, nx=nx, dy=gridThickM, ny=ny) + [[-LengthM/2.], [-ThickM]]

C = fp.CellVariable(mesh=mesh, name="$C$", value=0.0, hasOld=True)

viewer = fp.MatplotlibViewer(vars=C, datamin=0.0, datamax=cB)
viewer.plot()

X, Y = mesh.faceCenters
C.constrain(cP, where=mesh.facesTop & (-mask/2. <= X) & (X < mask/2.))

## CORE OF THE CODE IS THE NEXT EQUATION
eq1 = fp.TransientTerm() == fp.DiffusionTerm(DP)

## (b)

Simulate diffusion doping for the above situation and plot the concentration profile near the surface using the script provided for 2D simulations. Qualitatively compare your result to the one obtained in part 1 (b).  Please explain the similarities and differences and why.

In [ ]:
dt = 100.0         # Time step (s)
finalTime = 36150. # From problem 1 (s)

In [ ]:
results = []
t = 0
while t <= finalTime:
    t=t+dt
    C.updateOld()
    eq1.solve(C, dt=dt)
    if t % 1800 == 0:
        results.append([t, C([[0.], [-OutputDist]],order=1)[0]])
        # In a plain Python script, we would write
        #   viewer.plot()
        # but that doesn't work in a Jupyter notebook
        clear_output(wait=True)
        display(viewer)

In [ ]:
results = fp.numerix.array(results)
fig = plt.figure()
plt.plot(results[..., 0], results[..., 1])
plt.xlabel("$t / s$")
plt.ylabel(r"$C / \mu m^3$")